In [ ]:
import numpy as np
from scipy import spatial
from tqdm.auto import tqdm
from matplotlib.ticker import FormatStrFormatter
import time
# animation
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.core.display import HTML
def immune_ranking(self, T=0.7, alpha=0.95):
    
    T, alpha = self.T, self.alpha
    
    # part1
    A = 1 / self.Y

    # part2
    dist_matrix1 = spatial.distance.cdist(self.Chrom, self.Chrom, metric='hamming')  
    similiar_matrix1 = dist_matrix1 < 1 - T 
    similiar_matrix2 = similiar_matrix1.sum(axis=1)  
    S = (similiar_matrix2 - 1) / (self.size_pop - 1) 
    self.FitV = alpha * A / A.sum() + (1 - alpha) * S / (S.sum() + 1e-5)
    return self.FitV
class IA_TSP():
    def __init__(self, func, n_dim, size_pop=50, max_iter=200, prob_mut=0.001, T=0.7, alpha=0.95):   
      self.func = self.func_transformer(func)
      assert size_pop % 2 == 0, 'size_pop must be even integer'
      self.size_pop = size_pop  # size of population
      self.max_iter = max_iter
      self.prob_mut = prob_mut  # probability of mutation
      self.len_chrom = n_dim
      self.n_dim = n_dim

      self.Chrom = None
      self.X = None  # shape = (size_pop, n_dim)
      self.Y_raw = None  # shape = (size_pop,) , value is f(x)
      self.Y = None  # shape = (size_pop,) , value is f(x) + penalty for constraint
      self.FitV = None  # shape = (size_pop,)

      # self.FitV_history = []
      self.generation_best_X = []
      self.generation_best_Y = []

      self.all_history_X = []
      self.all_history_Y = []
      self.all_history_FitV = []

      self.best_x, self.best_y = None, None

      self.crtbp()

      self.T, self.alpha = T, alpha

    def func_transformer(self, func):
      def func_transformed(X):
        return np.array([func(x) for x in X])
    
      return func_transformed

    def crtbp(self):
        # create the population
        tmp = np.random.rand(self.size_pop, self.len_chrom)
        self.Chrom = tmp.argsort(axis=1)
        return self.Chrom

    def chrom2x(self, Chrom):
        return Chrom

    def x2y(self):
        self.Y_raw = self.func(self.X)
        self.Y = self.Y_raw
        return self.Y

    def ranking(self):
   
      self.FitV = -self.Y

    def selection(self, tourn_size=3):
      aspirants_idx = np.random.randint(self.size_pop, size=(self.size_pop, tourn_size))
      aspirants_values = self.FitV[aspirants_idx]
      winner = aspirants_values.argmax(axis=1)  # winner index in every team
      sel_index = [aspirants_idx[i, j] for i, j in enumerate(winner)]
      self.Chrom = self.Chrom[sel_index, :]
      return self.Chrom

    def clonning(self):
      Chrom, size_pop, len_chrom = self.Chrom, self.size_pop, self.len_chrom
      for i in range(0, size_pop, 2):
          Chrom1, Chrom2 = self.Chrom[i], self.Chrom[i + 1]
          self.Chrom[i], self.Chrom[i + 1] = Chrom1, Chrom1

    def mutation(self):
        def reverse(individual):
          n1, n2 = np.random.randint(0, individual.shape[0] - 1, 2)
          if n1 >= n2:
              n1, n2 = n2, n1 + 1
          individual[n1:n2] = individual[n1:n2][::-1]
          return individual

        for i in range(self.size_pop):
            if np.random.rand() < self.prob_mut:
                self.Chrom[i] = reverse(self.Chrom[i])
        return self.Chrom

    def run(self, max_iter=None):
        self.max_iter = max_iter or self.max_iter
        for i in tqdm(range(self.max_iter)):
            Chrom_old = self.Chrom.copy()
            self.X = self.chrom2x(self.Chrom)
            self.Y = self.x2y()
            self.ranking()
            self.selection()
            self.clonning()
            self.mutation()

            # put parent and offspring together and select the best size_pop number of population
            self.Chrom = np.concatenate([Chrom_old, self.Chrom], axis=0)
            self.X = self.chrom2x(self.Chrom)
            self.Y = self.x2y()
            self.ranking()
            selected_idx = np.argsort(self.Y)[:self.size_pop]
            self.Chrom = self.Chrom[selected_idx, :]

            # record the best ones
            generation_best_index = self.FitV.argmax()
            self.generation_best_X.append(self.X[generation_best_index, :].copy())
            self.generation_best_Y.append(self.Y[generation_best_index])
            self.all_history_X.append(self.X.copy())
            self.all_history_Y.append(self.Y.copy())
            self.all_history_FitV.append(self.FitV.copy())

        global_best_index = np.array(self.generation_best_Y).argmin()
        self.best_x = self.generation_best_X[global_best_index]
        self.best_y = self.func(np.array([self.best_x]))
        return self.best_x, self.best_y

    ranking = immune_ranking
class FunctionTSP():
  def __init__(self, seed=None):
    if seed:
      np.random.seed(seed=seed)

    self.num_points = 0
    self.points_coordinate = None

  def create(self, num_points, flag=True):
      if flag:
        self.num_points = num_points
        self.points_coordinate = np.random.rand(num_points, 2)
      else:
        self.num_points += num_points
        self.points_coordinate = np.concatenate([self.points_coordinate, np.random.rand(num_points, 2)])

      distance_matrix = spatial.distance.cdist(
          self.points_coordinate, self.points_coordinate, metric='euclidean')

      def cal_total_distance(routine):
          num_points, = routine.shape
          return sum([distance_matrix[routine[i % self.num_points], routine[(i + 1) % self.num_points]] for i in range(self.num_points)])

      return self.num_points, self.points_coordinate, distance_matrix, cal_total_distance

def plot_TSP():
  fig = plt.figure(figsize=(16,6))
  ax = fig.subplots(1, 2)

  best_points_ = np.concatenate([best_points, [best_points[0]]])
  best_points_coordinate = points_coordinate[best_points_, :]
  arrow_scale = float(max(best_points_coordinate[:, 0]))/float(75)

  ax[0].set_title(f'CLONAL, Distance: {round(best_distance[0], 2)}')
  ax[0].scatter(best_points_coordinate[0, 0], best_points_coordinate[0, 1],  color='r', s=50)
  ax[0].scatter(best_points_coordinate[1:-1, 0], best_points_coordinate[1:-1, 1],  color='b', s=50)
  for i in range(0,len(best_points_coordinate[:, 0])-1):
    ax[0].arrow(best_points_coordinate[i, 0], best_points_coordinate[i, 1], (best_points_coordinate[i+1, 0] - best_points_coordinate[i, 0]), (best_points_coordinate[i+1, 1] - best_points_coordinate[i, 1]), head_width = arrow_scale, 
                color ='c', length_includes_head=True, zorder=10)

  ax[0].xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
  ax[0].yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
  ax[0].set_xlabel("Longitude")
  ax[0].set_ylabel("Latitude")


  ax[1].plot(ia_tsp.generation_best_Y, color ='b')
  ax[1].set_xlabel("Iteration")
  ax[1].set_ylabel("Distance")
  plt.show()

def plot_diff():
    fig = plt.figure(figsize=(16,6))
    ax = fig.subplots(1, 2)
    
    best_points_ = np.concatenate([best_points, [best_points[0]]])
    best_points_coordinate = points_coordinate[best_points_, :]
    arrow_scale = float(max(best_points_coordinate[:, 0]))/float(75)

    ax[0].set_title(f'CLONALG, Distance: {round(best_distance[0], 2)}')
    ax[0].scatter(best_points_coordinate[0, 0], best_points_coordinate[0, 1],  color='r', s=50)
    ax[0].scatter(best_points_coordinate[1:-1, 0], best_points_coordinate[1:-1, 1],  color='b', s=50)
    for i in range(0,len(best_points_coordinate[:, 0])-1):
        ax[0].arrow(best_points_coordinate[i, 0], best_points_coordinate[i, 1], (best_points_coordinate[i+1, 0] - best_points_coordinate[i, 0]), (best_points_coordinate[i+1, 1] - best_points_coordinate[i, 1]), head_width = arrow_scale, 
                color ='c', length_includes_head=True, zorder=10)

    ax[0].xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    ax[0].yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    ax[0].set_xlabel("Longitude")
    ax[0].set_ylabel("Latitude")
    
    dynamic_points_coordinate = points_coordinate[path, :]
    
    dynamic_points_coordinate = np.concatenate([dynamic_points_coordinate, [dynamic_points_coordinate[0]]])
    
    ax[1].set_title(f'Dynamic, Distance: {round(distance, 2)}')
    ax[1].scatter(dynamic_points_coordinate[0, 0], dynamic_points_coordinate[0, 1],  color='r', s=50)
    ax[1].scatter(dynamic_points_coordinate[1:-1, 0], dynamic_points_coordinate[1:-1, 1],  color='b', s=50)
    for i in range(0,len(dynamic_points_coordinate[:, 0])-1):
        ax[1].arrow(dynamic_points_coordinate[i, 0], dynamic_points_coordinate[i, 1], (dynamic_points_coordinate[i+1, 0] - dynamic_points_coordinate[i, 0]), (dynamic_points_coordinate[i+1, 1] - dynamic_points_coordinate[i, 1]), head_width = arrow_scale, 
                color ='c', length_includes_head=True, zorder=10)

    ax[1].xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    ax[1].yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    ax[1].set_xlabel("Longitude")
    ax[1].set_ylabel("Latitude")

In [ ]:
tsp = FunctionTSP()
num_points, points_coordinate, distance_matrix, cal_total_distance = tsp.create(num_points=20)


In [ ]:
%%time
ia_tsp = IA_TSP(func=cal_total_distance, n_dim=num_points, size_pop=100, max_iter=70, prob_mut=0.2,
                    T=0.7, alpha=0.95)
best_points, best_distance = ia_tsp.run()
print('best routine:', best_points, 'best_distance:', best_distance)

In [ ]:
plot_TSP()

In [ ]:
def anim():
  best_x_history = ia_tsp.generation_best_X
  best_y_history = ia_tsp.generation_best_Y

  fig2, ax2 = plt.subplots(1, 1)
  ax2.set_title('CLONAL', loc='center')
  line = ax2.plot(points_coordinate[:, 0], points_coordinate[:, 1],
                  marker='o', markerfacecolor='b', color='c', linestyle='-')
  ax2.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
  ax2.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
  ax2.set_xlabel("Longitude")
  ax2.set_ylabel("Latitude")
  plt.ion()

  def update_scatter(frame):
      ax2.set_title(f'CLONAL, iter = {str(frame)},  distance = {round(best_y_history[frame], 2)}')
      points = best_x_history[frame]
      points = np.concatenate([points, [points[0]]])
      point_coordinate = points_coordinate[points, :]
      plt.setp(line, 'xdata', point_coordinate[:, 0], 'ydata', point_coordinate[:, 1])
      return line

  ani = FuncAnimation(fig2, update_scatter, blit=True, interval=25, frames=len(best_x_history))
  return ani


In [ ]:
ani = anim()

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
num_points, points_coordinate, distance_matrix, cal_total_distance = tsp.create(num_points=3, flag=False)
ia_tsp = IA_TSP(func=cal_total_distance, n_dim=num_points, size_pop=100, max_iter=150, prob_mut=0.2,
                    T=0.7, alpha=0.95)
best_points, best_distance = ia_tsp.run()
print('best routine:', best_points, 'best_distance:', best_distance)


In [ ]:
from python_tsp.exact import solve_tsp_dynamic_programming


In [ ]:
%%time

path,distance = solve_tsp_dynamic_programming(distance_matrix)

In [ ]:
print(path,distance)


In [ ]:
plot_diff()

In [ ]:
distance_matrix